In [1]:
from tdc.multi_pred import DrugRes
data = DrugRes(name = 'GDSC2')
df = data.get_data()
splits = data.get_split()

Found local copy...
Loading...
Done!


In [47]:
from rdkit.Chem import AllChem
from rdkit.Chem import MolFromSmiles
import numpy as np
from sklearn.decomposition import FastICA  
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelBinarizer

In [48]:
dataa = LabelBinarizer().fit_transform(np.asanyarray(df['Cell Line_ID']).reshape(-1,1))

In [58]:
df

,Drug_ID,Drug,Cell Line_ID,Cell Line,Y
0,Camptothecin,CC[C@@]1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=...,HCC1954,"[8.54820830373167, 2.5996072676336297, 10.3759...",-0.251083
1,Camptothecin,CC[C@@]1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=...,HCC1143,"[7.58193774904993, 2.81430257671695, 10.363326...",1.343315
2,Camptothecin,CC[C@@]1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=...,HCC1187,"[9.013252540641961, 2.9520929896608, 9.3474286...",1.736985
3,Camptothecin,CC[C@@]1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=...,HCC1395,"[7.4351511634642105, 2.8325700611437004, 10.34...",-2.309078
4,Camptothecin,CC[C@@]1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=...,HCC1599,"[8.334239608034789, 2.7477031637484997, 10.314...",-3.106684
...,...,...,...,...,...
92698,JQ1,CC1=C(SC2=C1C(=N[C@H](C3=NN=C(N32)C)CC(=O)OC(C...,EFM-192A,"[7.90969861787306, 3.0665091537456, 11.3513791...",3.576583
92699,JQ1,CC1=C(SC2=C1C(=N[C@H](C3=NN=C(N32)C)CC(=O)OC(C...,HCC1428,"[7.241512102682691, 2.7729214122229098, 9.7214...",1.402466
92700,JQ1,CC1=C(SC2=C1C(=N[C@H](C3=NN=C(N32)C)CC(=O)OC(C...,HDQ-P1,"[8.59362481381391, 2.7654211455101003, 9.91057...",2.762460
92701,JQ1,CC1=C(SC2=C1C(=N[C@H](C3=NN=C(N32)C)CC(=O)OC(C...,JIMT-1,"[8.44162845293353, 2.6392762542455497, 11.4637...",3.442930


In [46]:
pd.DataFrame(dataa).to_csv('celllines_onehot.csv', index=False)

In [51]:
dataa = MultiLabelBinarizer().fit_transform(np.asanyarray(df['Cell Line_ID'])).reshape(-1,1)

In [52]:
cells = dataa

In [5]:
cells = np.zeros((len(df['Cell Line']),len(df['Cell Line'].iloc[0])))
for i in range(len(df['Cell Line'])):
    cells[i] = df['Cell Line'].iloc[i]

In [54]:
morgans = pd.read_csv('morgans.csv')

In [7]:
morgans = []
for i in range(len(df['Drug'])):
    ecfp = AllChem.GetMorganFingerprintAsBitVect(MolFromSmiles(df.iloc[i]['Drug']), 3, nBits=512,useFeatures=True,
                                             useChirality=True).ToBitString()
    ecfp = np.fromstring(ecfp,'u1') - ord('0')
    morgans.append(ecfp)

In [55]:
y = np.asarray(df['Y'])
cells = np.asarray(cells)
morgans = np.asarray(morgans)

In [57]:
kf = KFold(n_splits=10,random_state=420, shuffle=True)
kf.get_n_splits(morgans)

print(kf)
fold = 0
for train_index, test_index in kf.split(morgans):
    print("TRAIN:", train_index, "TEST:", test_index)
#     morgan_train, morgan_test = morgans[train_index], morgans[test_index]
    cells_train, cells_test = cells[train_index], cells[test_index]
#     pca_obj = FastICA(n_components=256).fit(cells_train)
#     pcad_cells_train = pca_obj.transform(cells_train)
#     pcad_cells_test = pca_obj.transform(cells_test)
    y_train, y_test = y[train_index], y[test_index]
    
#     pd.DataFrame(morgan_train).to_csv('sparse/morgan_'+str(fold)+'_train.csv',index=False)
    pd.DataFrame(cells_train).to_csv('default/binarized_cells_'+str(fold)+'_train.csv',index=False)
#     pd.DataFrame(y_train).to_csv('sparse/y_train_'+str(fold)+'.csv',index=False)
    
#     pd.DataFrame(morgan_test).to_csv('sparse/morgan_'+str(fold)+'_test.csv',index=False)
    pd.DataFrame(cells_test).to_csv('default/binarized_cells_'+str(fold)+'_test.csv',index=False)
#     pd.DataFrame(y_test).to_csv('sparse/y_test_'+str(fold)+'.csv',index=False)
    fold = fold + 1
    


KFold(n_splits=10, random_state=420, shuffle=True)
TRAIN: [    0     1     2 ... 92699 92700 92702] TEST: [   17    29    38 ... 92692 92696 92701]
TRAIN: [    0     1     2 ... 92700 92701 92702] TEST: [    7    21    48 ... 92687 92689 92699]
TRAIN: [    2     5     6 ... 92700 92701 92702] TEST: [    0     1     3 ... 92679 92688 92698]
TRAIN: [    0     1     2 ... 92700 92701 92702] TEST: [   16    22    23 ... 92677 92683 92693]
TRAIN: [    0     1     2 ... 92700 92701 92702] TEST: [   19    24    63 ... 92682 92690 92697]
TRAIN: [    0     1     3 ... 92700 92701 92702] TEST: [    2     6     8 ... 92674 92675 92684]
TRAIN: [    0     1     2 ... 92700 92701 92702] TEST: [    9    11    20 ... 92648 92685 92695]
TRAIN: [    0     1     2 ... 92699 92700 92701] TEST: [   10    12    35 ... 92657 92694 92702]
TRAIN: [    0     1     2 ... 92699 92701 92702] TEST: [    5    18    28 ... 92665 92672 92700]
TRAIN: [    0     1     2 ... 92700 92701 92702] TEST: [   13    15    30 ..

In [ ]:
morgan_train.shape

In [30]:
print(np.fromstring(ecfp,'u1') - ord('0'))

[0 0 0 ... 0 0 0]


<ipython-input-30-fad640ecced2>:1: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  print(np.fromstring(ecfp,'u1') - ord('0'))
